In [5]:
import pandas as pd
import geopandas as gpd
import h3
import random
import os
from shapely.geometry import Point, Polygon

In [6]:
# Constants
NUM_POINTS = 9999  # Number of points
RESOLUTION = 9  # H3 resolution level

# Load the shapefile
GDF = gpd.read_file("./shapefiles/zh_poly.shp")
GDF = GDF.to_crs("EPSG:4326")

# Get the polygon
polygon = GDF.geometry.unary_union

In [7]:
def generate_random_location(polygon):
    """
    Generate a random location within the specified polygon.
    
    Parameters:
    polygon (shapely.geometry.Polygon): The polygon within which to generate the point.
    
    Returns:
    tuple: A tuple containing the latitude and longitude.
    """
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        lat = random.uniform(miny, maxy)
        lon = random.uniform(minx, maxx)
        point = Point(lon, lat)
        if polygon.contains(point):
            return lat, lon

In [8]:
# Generate mock data for H3 hexagons
data = [{
    "hex9": h3.geo_to_h3(*generate_random_location(polygon), RESOLUTION),
    "value": random.randint(1, 9999)  # Random value between 1 and 9999
} for _ in range(NUM_POINTS)]

KeyboardInterrupt: 

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
# csv_path = "hex/h3_farm_mock_data.csv"
csv_path = "hex/h3_indices_7.csv"

# Create directory if it doesn't exist
if not os.path.exists('hex'):
    os.makedirs('hex')

# Now you can save the file
df.to_csv(csv_path, index=False)

In [9]:
df = pd.read_csv("hex/h3_indices_7.csv")

# Convert H3 indices to polygons and create a GeoDataFrame
geometries = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in df["hex9"]]
gdf = gpd.GeoDataFrame(df, geometry=geometries)

# Save to a shapefile
shapefile_path = "app_data/h3_indices_7.shp"
gdf.to_file(shapefile_path)